In [284]:
from glob import glob
import os
import numpy as np
import pandas as pd
import sys
import re

import nltk
nltk.download('punkt') # Download the Punkt tokenizer models if you haven't already


[nltk_data] Downloading package punkt to /Users/bmd1905/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Data

In [288]:
file_paths_train = glob('data/raw/new train/*/*.txt') 
file_paths_test = glob('data/raw/new test/*/*.txt')
file_paths = file_paths_train + file_paths_test
len(file_paths)

26451

In [289]:
data = []
count = 0
for path in tqdm(file_paths):
  with open(path,'rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16")
    contents = contents.replace("\r\n",". ")
    sentences = nltk.sent_tokenize(contents)

    data.extend(sentences)

# Shuffle
import random

random.shuffle(data)

100%|██████████| 26451/26451 [00:10<00:00, 2572.77it/s]


In [290]:
len(data)

653815

In [295]:
from tqdm import tqdm

data_processed = []

# Use tqdm to create a progress bar
for content in tqdm(data):
    if len(content) > 50:
        content = content.replace('..', '.'). \
                            replace('"', ''). \
                            replace(': .', '.'). \
                            replace('. .', '.'). \
                            replace("'", ''). \
                            replace('“', ''). \
                            replace('”', '')
        
        # Check length of words
        words = content.split()
        for word in words:
            if len(word) > 10:
                content = None

        if content == None:
            continue

        if content[0] == '-':
            content = content.replace('-', '')

        if content[0] == ' ':
            content = content[1:]

        if '. - ' in content:
            content = content.replace('. - ', '. ')

        if '.. ' in content:
            content = content.replace('.. ', '... ')

        if '?.' in content:
            content = content.replace('?.', '?')

        if ' ?.' in content:
            content = content.replace(' ?.', '?')

        if '&' in content:
            continue

        if '( ' in content:
            content = content.replace('( ', '(')

        if ' )' in content:
            content = content.replace(' )', ')')
        
        if '()' in content:
            content = content.replace('()', '')

        if content[-2:] == ' .':
            content = content.replace(content[-2:], '.')
            #print(content)

        if '… ' in content:
            content = content.replace('… ', '... ')

        if '….' in content:
            content = content.replace('….', '...')

        if '…' in content:
            continue

        if ':.' in content:
            continue

        if content.isascii():
            continue

        if '  ' in content:
            content = content.replace('  ', ' ')

        match = re.search(r"\.\s\d+\.$", content)
        if match:
            content = re.sub(r"\.\s\d+\.$", ".", content)

        if content[0] == '·':
            continue


        data_processed.append(content)


100%|██████████| 653815/653815 [00:03<00:00, 208002.54it/s]


In [296]:
len(data_processed)

369673

In [297]:
count = 0
for index, sentence in enumerate(data_processed[120000:]):
  print(index, sentence)
  print("------")

  count += 1

  if count == 500:
    break


0 Nhưng bù lại, Năm Lúa đang sở hữu một vẻ đẹp rất công tử, làm không ít đồng nghiệp phải mơ ước và ganh tị.Đan Trường - Anh Bo Ai cũng biết tên thân mật của Đan Trường là Anh Bo, nhưng không phải ai cũng rõ căn nguyên.
------
1 Vì vậy, Chương trình ban đêm thế nào thì ban ngày cũng diễn ra như thế.
------
2 Đêm qua, trên SVĐ ở Genoa, Roberto Baggio đã nói lời chia tay trong trận giao hữu hoà 1-1 giữa Italy và Tây Ban Nha.
------
3 Trong khi đó, Man.City không còn sát thủ Anelka, người đã ghi bàn duy nhất trong trận thắng hồi đầu mùa. Tuy nhiên, chính sự chênh lệch về cán cân lực lượng là cơ hội để HLV các đội bóng yếu thể hiện khả năng dùng người và tổ chức chiến thuật.
------
4 Trời Sài Gòn lạnh và mưa nhẹ nhưng không khí của liveshow thì hừng hực lửa. Chương trình diễn ra đúng với kịch bản mà đích thân Thanh Thảo viết, dù trước đó chưa hề có buổi tổng duyệt.
------
5 Dự án gần đây nhất Thuỳ thực hiện là Nước sạch cho Làng Chài - cung cấp những bình lọc nước cho các gia đình sống ở L

In [298]:
len(data_processed)

369673

In [299]:
# extract Latin- characters only
alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!/\"\',\-\.:;?_\(\)]+$'
training_data=[]

for sentence in data_processed:
    if re.match(alphabet, sentence.lower()):
        sentence = sentence.strip()

        training_data.append(sentence)
    
print(len(training_data))

359993


In [300]:
with open('data/data_27topics_processed.txt', 'w') as f:
    for sentence in training_data:
        f.write(sentence)
        f.write('\n')